In [ ]:
# 라이브러리 로드
import pandas as pd
import numpy as np
import os

In [ ]:
label_list = os.listdir('/content/drive/MyDrive/뉴스분류')

In [ ]:
company_label = [list.replace('_predicted_results_hope.csv', '') for list in os.listdir('/content/drive/MyDrive/뉴스분류')]

In [ ]:
os.listdir('/content/drive/MyDrive/news_clean')

['CJ_clean.csv',
 'DB손해보험_clean.csv',
 'BGF리테일_clean.csv',
 'DL이앤씨_clean.csv',
 'DGB금융지주_clean.csv',
 'DL_clean.csv',
 'CJ대한통운_clean.csv',
 'BNK금융지주_clean.csv',
 'GS_clean.csv',
 'GS건설_clean.csv',
 'HD한국조선해양_clean.csv',
 'GS리테일_clean.csv',
 'HD현대미포_clean.csv',
 'HD현대_clean.csv',
 'HD현대인프라코어_clean.csv',
 'F&F_clean.csv',
 'GKL_clean.csv',
 'HD현대중공업_clean.csv',
 'HMM_clean.csv',
 'HL만도_clean.csv',
 'HS효성첨단소재_clean.csv',
 'HD현대일렉트릭_clean.csv',
 'JB금융지주_clean.csv',
 'KCC_clean.csv',
 'KG모빌리티_clean.csv',
 'KB금융_clean.csv',
 'KT&G_clean.csv',
 'KT_clean.csv',
 'LG디스플레이_clean.csv',
 'LG생활건강_clean.csv',
 'LG에너지솔루션_clean.csv',
 'LG_clean.csv',
 'LG이노텍_clean.csv',
 'LG유플러스_clean.csv',
 'LG전자_clean.csv',
 'NAVER_clean.csv',
 'LS_clean.csv',
 'LX인터내셔널_clean.csv',
 'NH투자증권_clean.csv',
 'LIG넥스원_clean.csv',
 'OCI_clean.csv',
 'LS ELECTRI

In [ ]:
company_list = [list.replace('_clean.csv', '') for list in os.listdir('/content/drive/MyDrive/news_clean')]

In [ ]:
for company in company_list:
  if company not in company_label:
    print(company)

삼성SDI


In [ ]:
len(os.listdir('/content/drive/MyDrive/뉴스분류'))

200

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/뉴스분류/BGF리테일_predicted_results_hope.csv', dtype={'ticker':str})

In [ ]:
df['일자'] = pd.to_datetime(df['일자'].astype(str), format='%Y%m%d')

In [ ]:
df.rename(columns={'일자':'date', 'pred_label':'label'}, inplace=True)

In [ ]:

def generate_news_features(news_df, k=5):
    """
    news_df: 컬럼 [date, ticker, label] (label: 1(호재), -1(악재), 0(중립))
    """
    # 1. 일별/종목별 기사 개수 집계 (P, N, Z)
    counts = news_df.groupby(['date', 'ticker', 'label']).size().unstack(fill_value=0)

    # 컬럼 이름 보장 (-1, 0, 1이 없는 경우 대비)
    for col in [-1, 0, 1]:
        if col not in counts.columns:
            counts[col] = 0

    counts = counts.rename(columns={1: 'P', -1: 'N', 0: 'Z'})
    counts['T'] = counts['P'] + counts['N'] + counts['Z']

    # 2. 핵심 피처 계산
    # sentiment: (P-N) / (T+k)
    counts['news_sentiment'] = (counts['P'] - counts['N']) / (counts['T'] + k)

    # conviction: |P-N| / (T+k)
    counts['news_conviction'] = (counts['P'] - counts['N']).abs() / (counts['T'] + k)

    # 3. 시계열 피처 (EWM) - 종목별로 계산
    # 날짜 순서 정렬 필수
    counts = counts.sort_index(level=['ticker', 'date'])
    counts['news_sentiment_ewm5'] = counts.groupby('ticker')['news_sentiment'].transform(
        lambda x: x.ewm(span=5, min_periods=1).mean()
    )

    # 4. 중요: T-1 Lag (오늘의 매매에는 어제의 뉴스까지만 반영)
    # 랭킹 엔진(L6R)에 들어가기 전, 뉴스 날짜를 하루 뒤로 밀어서 결합 준비
    final_features = counts[['news_sentiment', 'news_conviction', 'news_sentiment_ewm5']].reset_index()

    return final_features

# --- 적용 예시 ---
# dataset_daily = pd.read_csv('your_dataset_daily.csv') # 기존 데이터셋
# news_features = generate_news_features(raw_news_data)

# dataset_daily와 결합 (left join)
# dataset_daily.date와 news_features.date_for_merge를 키로 결합합니다.
# res = pd.merge(dataset_daily, news_features, left_on=['date', 'ticker'], right_on=['date_for_merge', 'ticker'], how='left')

In [ ]:
score = generate_news_features(df)

In [ ]:
date_index = pd.date_range(start='2015-01-30', end='2024-12-31', freq='D')

In [ ]:
date_index

DatetimeIndex(['2015-01-30', '2015-01-31', '2015-02-01', '2015-02-02',
               '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06',
               '2015-02-07', '2015-02-08',
               ...
               '2024-12-22', '2024-12-23', '2024-12-24', '2024-12-25',
               '2024-12-26', '2024-12-27', '2024-12-28', '2024-12-29',
               '2024-12-30', '2024-12-31'],
              dtype='datetime64[ns]', length=3624, freq='D')

In [ ]:
df_dates = pd.DataFrame(date_index, columns=['date'])

In [ ]:
score

label,date,ticker,news_sentiment,news_conviction,news_sentiment_ewm5
0,2015-01-11,282330,0.000000,0.000000,0.000000
1,2015-01-12,282330,0.000000,0.000000,0.000000
2,2015-01-14,282330,0.300000,0.300000,0.142105
3,2015-01-22,282330,0.000000,0.000000,0.083077
4,2015-01-26,282330,0.000000,0.000000,0.051185
...,...,...,...,...,...
770,2024-11-14,282330,0.000000,0.000000,0.076231
771,2024-11-15,282330,0.000000,0.000000,0.050821
772,2024-12-02,282330,0.000000,0.000000,0.033880
773,2024-12-18,282330,0.125000,0.125000,0.064254


In [ ]:
merge_df = pd.merge(df_dates, score, on='date', how='left')

In [ ]:
merge_df['ticker'] = '282330'

In [ ]:
merge_df.fillna(0, inplace=True)

In [ ]:
merge_df

,date,ticker,news_sentiment,news_conviction,news_sentiment_ewm5
0,2015-01-30,282330,0.0,0.0,0.0
1,2015-01-31,282330,0.0,0.0,0.0
2,2015-02-01,282330,0.0,0.0,0.0
3,2015-02-02,282330,0.0,0.0,0.0
4,2015-02-03,282330,0.0,0.0,0.0
...,...,...,...,...,...
3619,2024-12-27,282330,0.0,0.0,0.0
3620,2024-12-28,282330,0.0,0.0,0.0
3621,2024-12-29,282330,0.0,0.0,0.0
3622,2024-12-30,282330,0.0,0.0,0.0


In [ ]:
company_list = os.listdir('/content/drive/MyDrive/뉴스분류')

In [ ]:
company_list

['CJ_predicted_results_hope.csv',
 'DB손해보험_predicted_results_hope.csv',
 'BGF리테일_predicted_results_hope.csv',
 'DL이앤씨_predicted_results_hope.csv',
 'DGB금융지주_predicted_results_hope.csv',
 'DL_predicted_results_hope.csv',
 'CJ대한통운_predicted_results_hope.csv',
 'BNK금융지주_predicted_results_hope.csv',
 'GS_predicted_results_hope.csv',
 'GS건설_predicted_results_hope.csv',
 'HD한국조선해양_predicted_results_hope.csv',
 'GS리테일_predicted_results_hope.csv',
 'HD현대미포_predicted_results_hope.csv',
 'HD현대_predicted_results_hope.csv',
 'HD현대인프라코어_predicted_results_hope.csv',
 'F&F_predicted_results_hope.csv',
 'GKL_predicted_results_hope.csv',
 'HD현대중공업_predicted_results_hope.csv',
 'HMM_predicted_results_hope.csv',
 'HL만도_predicted_results_hope.csv',
 'HS효성첨단소재_predicted_results_hope.csv',
 'HD현대일렉트릭_predicted_results_hope.csv',
 'JB금융지주_predicted_results_hope.csv',
 'KCC_predicted_results_hope.csv',
 'K

In [ ]:
len(company_list)

200

In [ ]:
df_ticker = pd.read_csv('/content/drive/MyDrive/코스피200ticker.csv', dtype={'ticker':str})

In [ ]:
df_ticker.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323 entries, 0 to 322
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ticker      323 non-null    object
 1   start_date  323 non-null    object
 2   end_date    323 non-null    object
dtypes: object(3)
memory usage: 7.7+ KB


In [ ]:
df

,일자,제목,본문,ticker,pred_label,prob_neg,prob_neu,prob_pos,sentiment
0,20150108,"<생생코스피> DGB금융지주, 유상증자 확정발행가액 9010원",DGB금융지주는 유상증자 신주발행가액이 9010원으로 확정됐다고 8일 공시했다.,139130,0,0.0033,0.9951,0.0016,중립
1,20150108,"DGB금융지주, 유증 확정 발행가액 9010원",[뉴스핌=추연숙 기자] DGB금융지주는 유상증자 발행가액을 9010원으로 확정했다고...,139130,0,0.0046,0.9933,0.0020,중립
2,20150108,"DGB금융지주, 유상증자 신주 발행가액 9010원",[아시아경제 한진주 기자] DGB금융지주는 유상증자 신주 발행가액이 9010원으로 ...,139130,1,0.0002,0.0006,0.9992,호재(긍정)
3,20150108,"DGB금융지주, 유상증자 신주발행가액 9010원 확정",DGB금융지주는 유상증자 신주발행가액이 9010원으로 확정됐다고 8일 공시했다.,139130,0,0.0048,0.9936,0.0017,중립
4,20150108,[기업공시] DGB금융지주 外,"▲DGB금융지주=유상증자 신주발행가액 9,010원으로 확정 ▲슈넬생명과학=리젠메디컬...",139130,0,0.0151,0.9588,0.0262,중립
...,...,...,...,...,...,...,...,...,...
986,20241128,[포토] 금감원장 간담회 참석하는 최용호 DGB금융지주 의장,최용호 DGB금융지주 의장이 28일 서울 중구 은행회관 뱅커스클럽에서 열린 '금융감...,139130,0,0.0054,0.9933,0.0013,중립
987,20241203,"DGB금융지주, 지역사회와 상생 온실가스 배출량 공시 'ESG 금융' 정착",DGB금융그룹(회장 황병우 사진)이 ‘2024 한국의경영대상’을 수상했다. \n \...,139130,0,0.0170,0.9813,0.0017,중립
988,20241205,"LH 대구경북지역본부, DGB금융지주와 함께 ESG 세미나 성황리 개최",지난 4일 '2024년 대구 경북 ESG 추진협의체 2차 세미나'에 참석한 16개 ...,139130,0,0.0259,0.9726,0.0015,중립
989,20241220,[속보] 황병우 iM뱅크 은행장 연임 추천 DGB금융지주 회장 겸직 연장,황병우(57) DGB금융지주 회장이 iM뱅크 은행장 겸직 체제를 유지한다. \n \...,139130,0,0.0026,0.9937,0.0037,중립


In [ ]:
ticker = df['ticker'][0]

In [ ]:
ticker

'139130'

In [ ]:
df_ticker.loc[df_ticker['ticker'] == ticker]['start_date'].values[0]

'2023-04-28'

In [ ]:
start_date = df_ticker[df_ticker['ticker'] == ticker]['start_date'].values[0]

In [ ]:
(pd.to_datetime(start_date) - pd.Timedelta(days=1)).strftime('%Y-%m-%d')

'2015-01-29'

In [ ]:
len(company_list)

200

In [ ]:
num = 1
for company in company_list:
  df = pd.read_csv(f'/content/drive/MyDrive/뉴스분류/{company}', dtype={'ticker':str})
  df['ticker'] = df['ticker'].str.zfill(6)
  df['일자'] = pd.to_datetime(df['일자'].astype(str), format='%Y%m%d')
  df.rename(columns={'일자':'date', 'pred_label':'label'}, inplace=True)
  score = generate_news_features(df)
  ticker = df['ticker'][0]
  start_date = df_ticker[df_ticker['ticker'] == ticker]['start_date'].values[0]
  start_date = (pd.to_datetime(start_date) - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
  end_date = df_ticker[df_ticker['ticker'] == ticker]['end_date'].values[0]
  date_index = pd.date_range(start=start_date, end=end_date, freq='D')
  df_dates = pd.DataFrame(date_index, columns=['date'])
  merge_df = pd.merge(df_dates, score, on='date', how='left')
  merge_df['ticker'] = ticker
  merge_df.fillna(0, inplace=True)
  if len(merge_df) == 0:
    print(company)
  new_company = company.replace("_predicted_results_hope.csv", "")
  merge_df.to_csv(f'/content/drive/MyDrive/뉴스점수/{new_company}.csv', index=False, encoding="utf-8-sig")
  print(num, new_company)
  num += 1

1 CJ
2 DB손해보험
3 BGF리테일
4 DL이앤씨
5 DGB금융지주
6 DL
7 CJ대한통운
8 BNK금융지주
9 GS
10 GS건설
11 HD한국조선해양
12 GS리테일
13 HD현대미포
14 HD현대
15 HD현대인프라코어
16 F&F
17 GKL
18 HD현대중공업
19 HMM
20 HL만도
21 HS효성첨단소재
22 HD현대일렉트릭
23 JB금융지주
24 KCC
25 KG모빌리티
26 KB금융
27 KT&G
28 KT
29 LG디스플레이
30 LG생활건강
31 LG에너지솔루션
32 LG
33 LG이노텍
34 LG유플러스
35 LG전자
36 NAVER
37 LS
38 LX인터내셔널
39 NH투자증권
40 LIG넥스원
41 OCI
42 LS ELECTRIC
43 PI첨단소재
44 LG화학
45 OCI홀딩스
46 삼성에스디에스
47 POSCO홀딩스
48 삼성전기
49 S-Oil
50 SKC
51 SK케미칼
52 SK바이오팜
53 SK아이이테크놀로지
54 SK이노베이션
55 SK스퀘어
56 SK
57 SK네트웍스
58 SK바이오사이언스
59 강원랜드
60 TCC스틸
61 SK하이닉스
62 TKG휴켐스
63 금양
64 SK텔레콤
65 고려아연
66 금호타이어
67 기아
68 금호석유
69 대상
70 삼성전자
71 농심
72 녹십자
73 삼성중공업
74 녹십자홀딩스
75 기업으

In [ ]:
len(os.listdir('/content/drive/MyDrive/뉴스점수/'))

200

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/뉴스분류/삼성전자_predicted_results_hope.csv')

In [ ]:
df[df['일자'] == 20180211]

,일자,제목,본문,ticker,pred_label,prob_neg,prob_neu,prob_pos,sentiment
34280,20180211,삼성전자 총자산 창립 첫 300조원 돌파,삼성전자의 총자산이 처음 300조원을 넘어섰다. \n \n11일 업계에 따르면 지난...,5930,0,0.0147,0.9836,0.0016,중립
34281,20180211,공재광 시장 “평택시는 삼성전자와 함께 하겠다”,"“젊은 평택, 중단 없는 전진”을 올해 시정 목표로 정한 공재광 경기도 평택시장이 ...",5930,-1,0.9997,0.0002,0.0001,악재(부정)
34282,20180211,"대장주 삼성전자, '삼성코인'으로 전락?",국내 증시 대장주 삼성전자(005930)가 액면분할 선언 이후 너무 가벼워졌다. 주...,5930,0,0.1720,0.7700,0.0580,중립
34283,20180211,"삼성전자, 협력사 물품대금 4000억 조기 지급",삼성전자와 전자계열사가 설을 맞아 4000억원 규모의 협력업체 물품대금을 조기지급하...,5930,0,0.0097,0.9881,0.0023,중립
34284,20180211,"삼성전자 SK하이닉스 ""포항 지진에 반도체 공장 피해 없어""",삼성전자와 SK하이닉스는 11일 오전 경북 포항시에서 발생한 규모 4.6의 지진으로...,5930,-1,0.9999,0.0001,0.0000,악재(부정)
34285,20180211,삼성전자와 전자계열사 협력사에 물품대금 7일 앞당겨 지급,"삼성전자와 삼성디스플레이, 삼성SDI, 삼성전기, 삼성SDS 등 전자계열사들은 협력...",5930,0,0.1408,0.8529,0.0063,중립
34286,20180211,"삼성전자, 총자산 사상 첫 300조원 돌파 유형자산 증가 대기록 달성 견인",삼성전자의 총자산이 사상 처음으로 300조원을 돌파했다. 이는 우리나라 한해 예산(...,5930,1,0.0084,0.3898,0.6018,호재(긍정)
34287,20180211,"삼성전자, 지난해 총자산 300조원 첫 돌파",삼성전자의 2016년과 2017년 말 기준 총자산 규모. [자료=삼성전자 단위=조원...,5930,0,0.0154,0.9814,0.0032,중립
34288,20180211,삼성전자 `10년 주가` 교훈 바닥론 솔솔,올해 들어 삼성전자 주가가 10% 이상 하락하면서 '바닥론'이 고개를 들고 있다. ...,5930,-1,0.9999,0.0001,0.0000,악재(부정)
34289,20180211,"삼성전자, 총 자산 3백조 돌파 10년 만에 3배",삼성전자의 총자산이 3백조 원을 넘어서 10년 만에 3배로 늘었습니다. \n\n업계...,5930,0,0.0043,0.6378,0.3579,중립


In [ ]:
df['sentiment'].value_counts()

,count
sentiment,
중립,66034
호재(긍정),23436
악재(부정),20762


In [ ]:
company_list2 = os.listdir('/content/drive/MyDrive/뉴스점수')

In [ ]:
len(company_list2)

196

In [ ]:
num = 1
for company in company_list:
  df = pd.read_csv(f'/content/drive/MyDrive/뉴스분류/{company}', dtype={'ticker':str})
  df['ticker'] = df['ticker'].str.zfill(6)
  df['일자'] = pd.to_datetime(df['일자'].astype(str), format='%Y%m%d')
  df.rename(columns={'일자':'date', 'pred_label':'label'}, inplace=True)
  score = generate_news_features(df)
  ticker = df['ticker'][0]
  date_index = pd.date_range(start='2015-01-01', end='2024-12-31', freq='D')
  df_dates = pd.DataFrame(date_index, columns=['date'])
  merge_df = pd.merge(df_dates, score, on='date', how='left')
  merge_df['ticker'] = ticker
  merge_df.fillna(0, inplace=True)
  if len(merge_df) == 0:
    print(company)
  new_company = company.replace("_predicted_results_hope.csv", "")
  merge_df.to_csv(f'/content/drive/MyDrive/뉴스점수/{new_company}.csv', index=False, encoding="utf-8-sig")
  print(num, new_company)
  num += 1